In [1]:
import pandas as pd
from datetime import time
from ipyfilechooser import FileChooser
from IPython.display import display
import os

# 🔍 Fonction d'analyse Fichier Periodic
def analyze_behavior(file_path):
    df = pd.read_csv(file_path)
    
    # Afficher les événements uniques pour vérification
    print("Types d'événements dans les données :")
    print(df['Event'].unique())
    
    # Assurer format cohérent
    df['Datetime'] = pd.to_datetime(df['MM:DD:YYYY hh:mm:ss'], errors='coerce')
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce').dt.date
    df['Hour'] = pd.to_datetime(df['Hour'], format='%H:%M:%S', errors='coerce').dt.time
    
    # Trier les données par date et heure
    df = df.sort_values(['Date', 'Hour']).reset_index(drop=True)
    
    # Créer un identifiant de bloc pour les périodes consécutives
    # Cela va identifier chaque changement de période (Day->Night ou Night->Day)
    df['Period_Block'] = (df['Period'] != df['Period'].shift(1)).cumsum()
    
    # Définir les catégories d'événements
    specific_events = [
        'LeftDuringDispense',
        'RightDuringDispense',
        'LeftWithPellet',
        'RightWithPellet',
        'RightinTimeOut',
        'LeftinTimeOut'
    ]
    
    results = []
    
    # Regrouper par bloc de période (périodes consécutives)
    for (period_block), group in df.groupby('Period_Block'):
        # Le period_type est le même pour tout le groupe (Day ou Night)
        period_type = group['Period'].iloc[0]
        
        left_pokes = group[group['Event'] == 'Left'].shape[0]
        right_pokes = group[group['Event'] == 'Right'].shape[0]
        total_pokes = left_pokes + right_pokes
        correct = (group['Event'] == group['Active_Poke']).sum()
        
        # Comptage des pellets
        pellets = group[group['Event'] == 'Pellet'].shape[0]
        
        # First poke après Pellet
        pellet_indices = group[group['Event'] == 'Pellet'].index
        first_pokes = []
        for idx in pellet_indices:
            if idx + 1 in group.index:
                first_pokes.append(group.loc[idx + 1])
        first_pokes_df = pd.DataFrame(first_pokes)
        first_poke_total = len(first_pokes)
        first_poke_correct = (first_pokes_df['Event'] == first_pokes_df['Active_Poke']).sum() if not first_pokes_df.empty else 0
        
        # Calcul des pourcentages
        pct_first_poke_correct = round((first_poke_correct / first_poke_total) * 100, 2) if first_poke_total > 0 else None
        pct_correct = round((correct / total_pokes) * 100, 2) if total_pokes > 0 else None
        
        # Specific events
        spec_events = group[group['Event'].isin(specific_events)].shape[0]
        
        # Dates de début et fin du bloc
        start_date = group['Date'].min()
        end_date = group['Date'].max()
        
        results.append({
            'Period Block': period_block,
            'Period Type': period_type,
            'Start Date': start_date,
            'End Date': end_date,
            'Pellet': pellets,
            'Pokes': total_pokes,
            'Left Pokes': left_pokes,
            'Right Pokes': right_pokes,
            'Correct': correct,
            '% Correct': pct_correct,
            'First Poke': first_poke_total,
            'First Poke Correct': first_poke_correct,
            '% First Poke Correct': pct_first_poke_correct,
            'Specific Events': spec_events,
        })
    
    # Résumé
    summary_df = pd.DataFrame(results)
    
    # Sauvegarde CSV
    folder = os.path.dirname(file_path)
    base = os.path.splitext(os.path.basename(file_path))[0]
    output_summary = os.path.join(folder, f"{base}_global.csv")
    summary_df.to_csv(output_summary, index=False)
    
    print(f"\n✅ Fichier résumé sauvegardé avec succès !")
    print(f"📍 Chemin : {output_summary}")
    display(summary_df)

# 🎯 Sélection du fichier avec FileChooser
def choose_file_and_analyze():
    def on_file_selected(chooser):
        selected_file = chooser.selected
        print(f"\n📂 Fichier sélectionné : {selected_file}")
        analyze_behavior(selected_file)
    
    fc = FileChooser(".", show_only_dirs=False, title="<b>Sélectionnez le fichier .csv modifié</b>")
    display(fc)
    fc.register_callback(on_file_selected)

# 🔽 Lance l'interface
choose_file_and_analyze()

FileChooser(path='C:\Users\AudreyHay\Documents\Carla\Visual Code\HayLabAnalysis\python', filename='', title='<…

,Period Block,Period Type,Start Date,End Date,Pellet,Pokes,Left Pokes,Right Pokes,Correct,% Correct,First Poke,First Poke Correct,% First Poke Correct,Specific Events
0,1,Day,2025-04-30,2025-04-30,32,62,31,31,32,51.61,31,15,48.39,8
1,2,Night,2025-04-30,2025-05-01,133,289,166,123,133,46.02,132,68,51.52,40
2,3,Day,2025-05-01,2025-05-01,40,91,33,58,40,43.96,39,18,46.15,8
3,4,Night,2025-05-01,2025-05-02,104,171,74,97,104,60.82,104,60,57.69,30
4,5,Day,2025-05-02,2025-05-02,4,8,6,2,4,50.00,3,1,33.33,1


,Period Block,Period Type,Start Date,End Date,Pellet,Pokes,Left Pokes,Right Pokes,Correct,% Correct,First Poke,First Poke Correct,% First Poke Correct,Specific Events
0,1,Day,2025-04-30,2025-04-30,27,40,18,22,27,67.50,26,16,61.54,5
1,2,Night,2025-04-30,2025-05-01,146,217,96,121,146,67.28,145,91,62.76,78
2,3,Day,2025-05-01,2025-05-01,20,30,15,15,20,66.67,19,11,57.89,6
3,4,Night,2025-05-01,2025-05-02,121,173,75,98,121,69.94,121,84,69.42,115
4,5,Day,2025-05-02,2025-05-02,6,12,6,6,6,50.00,5,1,20.00,3
